# (1) 2.4: Evaluating Hyperparameters

In [2]:
!pip install scikeras

In [3]:
!pip install bayesian-optimization

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [5]:
# Set option to ensure charts are displayed inline in the notebook
%matplotlib inline
# Set option to display max number of columns
pd.set_option('display.max_columns', None)

In [20]:
path = r'C:\Users\nvak6\Machine Learning with Python'
pleasant = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
climate = pd.read_csv(os.path.join(path, 'weather_cleaned.csv'), index_col=False)

### Data Wrangling

In [23]:
# Drop DATE column from answers
pleasant.drop(columns = 'DATE', inplace = True)

In [25]:
pleasant.shape

(22950, 15)

In [29]:
climate.shape

(22950, 135)

In [33]:
# Turn X and answers from a df to arrays
X = np.array(climate)
y = np.array(pleasant)

In [35]:
X = X.reshape(-1,15,9)

In [37]:
X.shape

(22950, 15, 9)

In [39]:
# Use argmax to transform y
y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [41]:
# Check y layout
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

### Spliting the Data

In [44]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [46]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


### Bayesian Hyperparmater Optimization

In [49]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [51]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [53]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\nvak6\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nvak6\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
15/15 - 5s - 344ms/step - accuracy: 0.6008 - loss: 2.7109
Epoch 2/25
15/15 - 1s - 56ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 53ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 61ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 1s - 55ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 56ms/step - accuracy: 0.6440 - loss: 2.6895
Epoch 7/25
15/15 - 1s - 55ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 1s - 50ms/step - accuracy: 0.6440 - loss: 2.6854
Epoch 9/25
15/15 - 1s - 50ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 1s - 57ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 1s - 91ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 1s - 82ms/step - accuracy: 0.6440 - loss: 2.6784
Epoch 13/25
15/15 - 1s - 61ms/step - accuracy: 0.6440 - loss: 2.6768
Epoch 14/25
15/15 - 1s - 45ms/step - accuracy: 0.6440 - loss: 2.6753
Epoch 15/25
15/15 - 1s - 48ms/step - accur

C:\Users\nvak6\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nvak6\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
15/15 - 4s - 289ms/step - accuracy: 0.6017 - loss: 2.7026
Epoch 2/25
15/15 - 1s - 54ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 64ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 55ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 1s - 61ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 2s - 103ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 1s - 67ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 1s - 64ms/step - accuracy: 0.6440 - loss: 2.6853
Epoch 9/25
15/15 - 1s - 68ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 1s - 69ms/step - accuracy: 0.6440 - loss: 2.6817
Epoch 11/25
15/15 - 1s - 51ms/step - accuracy: 0.6440 - loss: 2.6800
Epoch 12/25
15/15 - 1s - 81ms/step - accuracy: 0.6440 - loss: 2.6784
Epoch 13/25
15/15 - 1s - 87ms/step - accuracy: 0.6440 - loss: 2.6768
Epoch 14/25
15/15 - 1s - 99ms/step - accuracy: 0.6440 - loss: 2.6753
Epoch 15/25
15/15 - 1s - 98ms/step - accu

...

C:\Users\nvak6\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nvak6\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 4s - 178ms/step - accuracy: 0.5879 - loss: 1.4331
Epoch 2/35
21/21 - 1s - 29ms/step - accuracy: 0.6568 - loss: 1.0238
Epoch 3/35
21/21 - 1s - 30ms/step - accuracy: 0.6954 - loss: 0.9270
Epoch 4/35
21/21 - 1s - 31ms/step - accuracy: 0.7142 - loss: 0.8569
Epoch 5/35
21/21 - 1s - 30ms/step - accuracy: 0.7264 - loss: 0.7987
Epoch 6/35
21/21 - 1s - 29ms/step - accuracy: 0.7431 - loss: 0.7523
Epoch 7/35
21/21 - 1s - 29ms/step - accuracy: 0.7551 - loss: 0.7129
Epoch 8/35
21/21 - 1s - 26ms/step - accuracy: 0.7686 - loss: 0.6769
Epoch 9/35
21/21 - 1s - 28ms/step - accuracy: 0.7772 - loss: 0.6448
Epoch 10/35
21/21 - 1s - 29ms/step - accuracy: 0.7869 - loss: 0.6103
Epoch 11/35
21/21 - 1s - 26ms/step - accuracy: 0.7968 - loss: 0.5780
Epoch 12/35
21/21 - 1s - 29ms/step - accuracy: 0.8105 - loss: 0.5409
Epoch 13/35
21/21 - 1s - 28ms/step - accuracy: 0.8200 - loss: 0.5085
Epoch 14/35
21/21 - 1s - 28ms/step - accuracy: 0.8308 - loss: 0.4745
Epoch 15/35
21/21 - 1s - 30ms/step - accuracy: 0.8380

C:\Users\nvak6\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\nvak6\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 - 5s - 236ms/step - accuracy: 0.5670 - loss: 1.4531
Epoch 2/35
21/21 - 1s - 31ms/step - accuracy: 0.6589 - loss: 1.0037
Epoch 3/35
21/21 - 1s - 30ms/step - accuracy: 0.6852 - loss: 0.9094
Epoch 4/35
21/21 - 1s - 30ms/step - accuracy: 0.7084 - loss: 0.8387
Epoch 5/35
21/21 - 1s - 30ms/step - accuracy: 0.7243 - loss: 0.7905
Epoch 6/35
21/21 - 1s - 29ms/step - accuracy: 0.7361 - loss: 0.7509
Epoch 7/35
21/21 - 1s - 32ms/step - accuracy: 0.7502 - loss: 0.7186
Epoch 8/35
21/21 - 1s - 28ms/step - accuracy: 0.7580 - loss: 0.6885
Epoch 9/35
21/21 - 1s - 31ms/step - accuracy: 0.7671 - loss: 0.6612
Epoch 10/35
21/21 - 1s - 28ms/step - accuracy: 0.7771 - loss: 0.6375
Epoch 11/35
21/21 - 1s - 32ms/step - accuracy: 0.7858 - loss: 0.6049
Epoch 12/35
21/21 - 1s - 32ms/step - accuracy: 0.7928 - loss: 0.5796
Epoch 13/35
21/21 - 1s - 32ms/step - accuracy: 0.8078 - loss: 0.5471
Epoch 14/35
21/21 - 1s - 30ms/step - accuracy: 0.8136 - loss: 0.5197
Epoch 15/35
21/21 - 1s - 30ms/step - accuracy: 0.8190

ValueError: Input y contains NaN.

In [56]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x2ba6b5bacc0>}

### CNN Model with Optimized Parameters

In [59]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = Adadelta(learning_rate=0.7631771981307285)  

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

C:\Users\nvak6\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [61]:
model.summary()

Model: "sequential_75"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_75 (Conv1D)              │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_380 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_381 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_382 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_75 (MaxPooling1D) │ (None, 7, 61)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_75 (Flatten)            │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_383 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [63]:
# Put the y_test set 
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [65]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [67]:
# Compile the model with categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [69]:
# Fit the model to the data
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 4s - 118ms/step - accuracy: 0.6195 - loss: 1.3414
Epoch 2/47
38/38 - 1s - 31ms/step - accuracy: 0.7011 - loss: 0.8838
Epoch 3/47
38/38 - 1s - 31ms/step - accuracy: 0.7276 - loss: 0.7950
Epoch 4/47
38/38 - 1s - 31ms/step - accuracy: 0.7463 - loss: 0.7402
Epoch 5/47
38/38 - 1s - 23ms/step - accuracy: 0.7625 - loss: 0.6954
Epoch 6/47
38/38 - 1s - 25ms/step - accuracy: 0.7701 - loss: 0.6639
Epoch 7/47
38/38 - 1s - 23ms/step - accuracy: 0.7826 - loss: 0.6316
Epoch 8/47
38/38 - 1s - 27ms/step - accuracy: 0.7907 - loss: 0.6038
Epoch 9/47
38/38 - 1s - 20ms/step - accuracy: 0.7960 - loss: 0.5843
Epoch 10/47
38/38 - 1s - 20ms/step - accuracy: 0.8071 - loss: 0.5566
Epoch 11/47
38/38 - 1s - 26ms/step - accuracy: 0.8139 - loss: 0.5361
Epoch 12/47
38/38 - 1s - 38ms/step - accuracy: 0.8205 - loss: 0.5112
Epoch 13/47
38/38 - 1s - 37ms/step - accuracy: 0.8237 - loss: 0.4973
Epoch 14/47
38/38 - 1s - 29ms/step - accuracy: 0.8345 - loss: 0.4718
Epoch 15/47
38/38 - 1s - 28ms/step - accur

### Confusion Matrix

In [72]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [74]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [76]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [78]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3533        97        12       6           7         5       1   
BELGRADE       73       979        11       6           1         3       0   
BUDAPEST       18        12       140      12           2        14       0   
DEBILT          5         3         5      62           4         2       0   
DUSSELDORF      2         0         0       4          13         7       0   
HEATHROW       10         0         0       2           5        54       0   
KASSEL          0         0         1       0           0         0       5   
LJUBLJANA       5         3         0       0           0         0       0   
MAASTRICHT      5         0         0       0           0         1       0   
MADRID         46        10         2       0           1         8       1   
MUNCHENB        6         0         0       0       